In [28]:
import glob
import itertools
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy.spatial import cKDTree
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

BASE_PATH = os.path.expanduser("~/Code/ModelQuake")
DATA_PATH = os.path.join(BASE_PATH, "data")
if not os.path.exists(DATA_PATH): os.makedirs(DATA_PATH)
DATA_FILE = os.path.join(BASE_PATH, "data/database.csv")
if not os.path.isfile(DATA_FILE): raise Exception()
SAVES_PATH = os.path.join(BASE_PATH, "saves")
if not os.path.exists(SAVES_PATH): os.makedirs(SAVES_PATH)
METRICS_FILE = os.path.join(BASE_PATH, "saves/all.metrics")
if not os.path.isfile(METRICS_FILE):
	with open(METRICS_FILE, "w") as f:
		f.write("[]")

WINDOW_RADIUS = 15
WINDOW_INTERVAL = 30 * 24 * 60 * 60

# PROCESSING PHASE
class Dataset(Dataset):
	def __init__(self, X, y):
		self.X = torch.tensor(X, dtype=torch.float32)
		self.y = torch.tensor(y, dtype=torch.float32)
	def __getitem__(self, i): return self.X[i], self.y[i]
	def __len__(self): return len(self.X)

def process_data():
	df = pd.read_csv(DATA_FILE)
	df["timestamp"] = pd.to_datetime(df["Date"] + " " + df["Time"], errors="coerce")
	df = df.dropna(subset=["timestamp", "Latitude", "Longitude", "Depth", "Magnitude"])
	df["timestamp"] = df["timestamp"].astype(np.int64) // 10**9

	X = df[["timestamp", "Latitude", "Longitude", "Depth", "Magnitude"]].values
	y = np.zeros(len(df), dtype=np.float32)

	tree = cKDTree(df[["Latitude", "Longitude"]])
	timestamps = df["timestamp"].values
	for i, (lat, lon, ts) in enumerate(zip(df["Latitude"], df["Longitude"], timestamps)):
		indices = tree.query_ball_point([lat, lon], WINDOW_RADIUS / 6371 * (180 / np.pi))
		for j in indices:
			if i != j and ts < timestamps[j] < ts + WINDOW_INTERVAL:
				y[i] = 1
				break
	return X, y

X, y = process_data()

# SELECTION PHASE
class BaseModel(nn.Module):
	def __init__(self, id):
		super().__init__()
		self.id = id
	def forward(self, x): raise NotImplementedError()

class RNNLSTMModel(BaseModel):
	def __init__(self, input_dim, lstm_hidden_dim, num_layers, dropout, lr, weight_decay):
		super().__init__(f"RNNLSTM({input_dim}-{lstm_hidden_dim}-{num_layers}-{dropout})-{lr}-{weight_decay}")
		self.lstm = nn.LSTM(input_dim, lstm_hidden_dim, num_layers, batch_first=True, dropout=dropout)
		self.fc = nn.Linear(lstm_hidden_dim, 1)
	def forward(self, x):
		x = x.unsqueeze(1)
		_, (h_n, _) = self.lstm(x)
		output = self.fc(h_n[-1])
		return torch.sigmoid(output).squeeze(-1)

class MLPModel(BaseModel):
	def __init__(self, input_dim, hidden_dim, num_layers, dropout, lr, weight_decay):
		super().__init__(f"MLP({input_dim}-{hidden_dim}-{num_layers}-{dropout})-{lr}-{weight_decay}")

		layers = []
		for _ in range(num_layers):
			layers.append(nn.Linear(input_dim if len(layers) == 0 else hidden_dim, hidden_dim))
			layers.append(nn.ReLU())
			layers.append(nn.Dropout(dropout))
		layers.append(nn.Linear(hidden_dim, 1))

		self.network = nn.Sequential(*layers)
	def forward(self, x):
		output = self.network(x)
		return torch.sigmoid(output).squeeze(-1)

def instantiate(model_type, **kwargs):
	if model_type == "RNNLSTM": return RNNLSTMModel(**kwargs)
	elif model_type == "MLP": return MLPModel(**kwargs)
	else: raise ValueError(f"Unknown model type {model_type}")

def save(model, epoch, train_loss, val_loss):
	model_folder = os.path.join(SAVES_PATH, model.id)
	os.makedirs(model_folder, exist_ok=True)
	torch.save(model.state_dict(), os.path.join(model_folder, f"epoch{epoch}.pth"))
	with open(os.path.join(model_folder, f"epoch{epoch}.losses"), "w") as f:
		json.dump({"train": train_loss, "val": val_loss}, f)

def load(model, epoch=None):
	model_folder = os.path.join(SAVES_PATH, model.id)
	model_files = sorted(glob.glob(os.path.join(model_folder, "*.pth")), key=os.path.getmtime)
	if not model_files: return
	model_checkpoint = model_files[-1] if epoch is None else os.path.join(model_folder, f"epoch{epoch}.pth")
	model.load_state_dict(torch.load(model_checkpoint, map_location=DEVICE))

def train(model, train_loader, val_loader, criterion, optimizer, epochs):
	best_val_loss = float("inf")
	patience_counter = 0

	for epoch in range(1, epochs + 1):
		model.train()
		train_loss = 0
		for X, y in train_loader:
			X = X.to(DEVICE)
			y = y.to(DEVICE)
			optimizer.zero_grad()
			output = model(X)
			loss = criterion(output, y)
			loss.backward()
			optimizer.step()
			train_loss += loss.item()
		train_loss /= len(train_loader)

		model.eval()
		val_loss = sum(criterion(model(X.to(DEVICE)), y.to(DEVICE)).item() for X, y in val_loader) / len(val_loader)
		print(f"{model.id} epoch {epoch}: train_loss = {train_loss:.5f}, val_loss = {val_loss:.5f}")
		save(model, epoch, train_loss, val_loss)
		if val_loss < best_val_loss:
			best_val_loss = val_loss
			patience_counter = 0
		else:
			patience_counter += 1
			if patience_counter >= 50:
				print("Stopping early")
				break

def evaluate(model, X_test, y_test):
	load(model)

	model.eval()
	with torch.no_grad():
		probabilities = model(torch.tensor(X_test, dtype=torch.float32).to(DEVICE)).squeeze().cpu().numpy()
	M = 0.5
	if not len(np.unique(y_test)) < 2: M = roc_auc_score(y_test, probabilities)
	return round(M, 8)

def crossvalidate(model, criterion, optimizer, X, y, id, folds, epochs):
	if folds == 1:
		π = int(0.8 * len(X))
		X_train, y_train = X[:π], y[:π]
		X_test, y_test = X[π:], y[π:]

		scaler = StandardScaler()
		X_train = scaler.fit_transform(X_train)
		X_test = scaler.transform(X_test)

		joblib.dump(scaler, os.path.join(SAVES_PATH, f"{id}/scaler.pkl"))

		υ = int(0.2 * len(X_train))
		train_loader = DataLoader(Dataset(X_train[υ:], y_train[υ:]), batch_size=64, shuffle=True)
		val_loader = DataLoader(Dataset(X_train[:υ], y_train[:υ]), batch_size=64)

		train(model, train_loader, val_loader, criterion, optimizer, epochs=epochs)
		M_list = [evaluate(model, X_test, y_test)]
	else:
		kf = KFold(n_splits=folds, shuffle=True, random_state=37)
		M_list = []
		best_scaler = None
		best_M = -float("inf")

		for train_i, test_i in kf.split(X):
			scaler = StandardScaler()
			X_train = scaler.fit_transform(X[train_i])
			X_test = scaler.transform(X[test_i])
			y_train = y[train_i]
			y_test = y[test_i]

			υ = int(0.2 * len(X_train))
			train_loader = DataLoader(Dataset(X_train[υ:], y_train[υ:]), batch_size=64, shuffle=True)
			val_loader = DataLoader(Dataset(X_train[:υ], y_train[:υ]), batch_size=64)

			train(model, train_loader, val_loader, criterion, optimizer, epochs=epochs)
			M = evaluate(model, X_test, y_test)
			if M > best_M:
				best_M = M
				best_scaler = scaler
			M_list.append(M)

	if best_scaler: joblib.dump(best_scaler, os.path.join(SAVES_PATH, f"{id}/scaler.pkl"))

	rounded_mean_M = round(np.mean(M_list), 8)
	json.dump(
		(json.load(open(METRICS_FILE)) if os.path.getsize(METRICS_FILE) > 0 else []) + [{"id": id, "M": rounded_mean_M}],
		open(METRICS_FILE, "w"),
		indent=2
	)

def gridsearch(model_type, grid, X, y, folds=1, epochs=5):
	metrics_dict = {}
	if os.path.getsize(METRICS_FILE) > 0:
		with open(METRICS_FILE) as f:
			metrics_dict = {metrics_entry["id"]: metrics_entry["M"] for metrics_entry in json.load(f)}

	for column in itertools.product(*grid.values()):
		kwargs = dict(zip(grid.keys(), column))
		model = instantiate(model_type, **kwargs).to(DEVICE)
		if model.id in metrics_dict: continue
		criterion = nn.BCELoss()
		optimizer = optim.Adam(model.parameters(), lr=kwargs["lr"], weight_decay=kwargs["weight_decay"])
		crossvalidate(model, criterion, optimizer, X, y, model.id, folds=folds, epochs=epochs)

grid = {
	"input_dim": [5],
	"lstm_hidden_dim": [64],
	"num_layers": [4],
	"dropout": [0.0],
	"lr": [0.001],
	"weight_decay": [0.0],
}
gridsearch("RNNLSTM", grid, X, y, folds=2, epochs=100)

# grid = {
# 	"input_dim": [5],
# 	"hidden_dim": [128],
# 	"num_layers": [4],
# 	"dropout": [0.1],
# 	"lr": [0.001],
# 	"weight_decay": [0.0],
# }
# gridsearch("MLP", grid, X, y)
# gridpredict("MLP", grid, query1)
# gridpredict("MLP", grid, query2)
# gridpredict("MLP", grid, query3)

# INFERENCE PHASE
def gridpredict(model_type, grid, query):
	query = np.array(query, dtype=np.float32).reshape(1, -1)
	for column in itertools.product(*grid.values()):
		model = instantiate(model_type, **dict(zip(grid.keys(), column))).to(DEVICE)
		scaler = joblib.load(os.path.join(SAVES_PATH, f"{model.id}/scaler.pkl"))
		query = torch.tensor(scaler.transform(query), dtype=torch.float32).to(DEVICE)
		load(model)

		model.eval()
		with torch.no_grad(): probability = model(query).item()
		print(f"{model.id}: {probability:.9f}")

gridpredict("RNNLSTM", grid, (1797531234, -15.5, -175.2, 600.0, 8.0))
gridpredict("RNNLSTM", grid, (1797531234, 1.0, 69.0, 300.0, 8.0))
gridpredict("RNNLSTM", grid, (1797531234, 20.0, 160.0, 500.0, 8.0))

gridpredict("RNNLSTM", grid, (1697531234, -15.5, -175.2, 600.0, 8.0))
gridpredict("RNNLSTM", grid, (1697531234, 1.0, 69.0, 300.0, 8.0))
gridpredict("RNNLSTM", grid, (1697531234, 20.0, 160.0, 500.0, 8.0))

gridpredict("RNNLSTM", grid, (1712345678, -15.5, -175.2, 600.0, 8.0))
gridpredict("RNNLSTM", grid, (1712345678, 1.0, 69.0, 300.0, 8.0))
gridpredict("RNNLSTM", grid, (1712345678, 20.0, 160.0, 500.0, 8.0))

RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 1: train_loss = 0.44284, val_loss = 0.29276
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 2: train_loss = 0.35069, val_loss = 0.28900
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 3: train_loss = 0.34937, val_loss = 0.29180
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 4: train_loss = 0.34863, val_loss = 0.28913
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 5: train_loss = 0.34843, val_loss = 0.28936
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 6: train_loss = 0.34722, val_loss = 0.28758
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 7: train_loss = 0.34710, val_loss = 0.29278
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 8: train_loss = 0.34709, val_loss = 0.28792
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 9: train_loss = 0.34578, val_loss = 0.28912
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 10: train_loss = 0.34544, val_loss = 0.28986
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 11: train_loss = 0.34675, val_loss = 0.28928
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 12: train_loss = 0.34525, val_loss = 0.28816
RNNLSTM(5-64-4-0.0)-0.001-0.0 epoch 1